Тема «POS-tagger и NER»
Задание
 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
сравнить все реализованные методы, сделать выводы  


Задание 2. Проверить, насколько хорошо работает NER
Данные брать из Index of /pub/named_entities
проверить NER из nltk/spacy/deeppavlov.
написать свой NER, попробовать разные подходы.
передаём в сетку токен и его соседей.
передаём в сетку только токен.
свой вариант.
сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


## 1 UnigramTagger, BigramTagger, TrigramTagger

In [12]:

!pip install pyconll -q
import pandas as pd
import numpy as np
import re
import string

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.corpus import brown

import pyconll
import nltk
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

In [2]:

%%capture
!wget -q -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget -q -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

In [3]:
full_train = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [4]:
[el.text for el in full_train][:5]

['Анкета.',
 'Начальник областного управления связи Семен Еремеевич был человек простой, приходил на работу всегда вовремя, здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом "Муха".',
 'В приемной его с утра ожидали посетители, - кое-кто с важными делами, а кое-кто и с такими, которые легко можно было решить в нижестоящих инстанциях, не затрудняя Семена Еремеевича.',
 'Однако стиль работы Семена Еремеевича заключался в том, чтобы принимать всех желающих и лично вникать в дело.',
 'Приемная была обставлена просто, но по-деловому.']

In [45]:
for sent in full_train[1:2]:
    for token in sent[:3]:
        print(token.form, token.upos)
    print()

Начальник NOUN
областного ADJ
управления NOUN



In [7]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 194
Наибольшая длина токена 31


In [8]:
# Заполним данными
data_train = []
for sent in full_train[:]:
    data_train.append([(token.form, token.upos) for token in sent])

data_test = []
for sent in full_test[:]:
    data_test.append([(token.form, token.upos) for token in sent]) # form - само слово  upos - форма

data_sent_test = []
for sent in full_test[:]:
    data_sent_test.append([token.form for token in sent])

In [43]:
for el in data_train[1:2]:
  print(el[3])

('связи', 'NOUN')


In [13]:
unigram_tagger = UnigramTagger(data_train)
unigram_acc = unigram_tagger.accuracy(data_test)

bigram_tagger = BigramTagger(data_train)
bigram_acc = bigram_tagger.accuracy(data_test)

trigram_tagger = TrigramTagger(data_train)
trigram_acc = trigram_tagger.accuracy(data_test)

# Комбинации
# 2+1
bigram_tagger = BigramTagger(data_train, backoff=unigram_tagger) # backoff=unigram_tagger - + unigram к bigram модели
bigram_unigram_acc = bigram_tagger.accuracy(data_test)
# 3+2+1
trigram_tagger = TrigramTagger(data_train, backoff=bigram_tagger)
trigram_bigram_unigram_acc = trigram_tagger.accuracy(data_test)

# 3+1
unigram_tagger_1 = UnigramTagger(data_train)
trigram_tagger_2 = TrigramTagger(data_train, backoff=unigram_tagger)
trigram_unigram_acc = trigram_tagger_2.accuracy(data_test)

print(f'Accuracy:\nUnigram Tagger: {round(unigram_acc, 5)},\nBigram Tagger: {round(bigram_acc, 5)},\n'
      f'Trigram Tagger: {round(trigram_acc, 5)},\nBigram and Unigram Tagger: {round(bigram_unigram_acc, 5)},\n'
      f'Trigram, Bigram and Unigram Tagger: {round(trigram_bigram_unigram_acc, 5)},\n'
      f'Trigram and Unigram Tagger: {round(trigram_unigram_acc, 5)},')

Accuracy:
Unigram Tagger: 0.82373,
Bigram Tagger: 0.60939,
Trigram Tagger: 0.17786,
Bigram and Unigram Tagger: 0.82928,
Trigram, Bigram and Unigram Tagger: 0.82914,
Trigram and Unigram Tagger: 0.82855,


In [14]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN')
tag = backoff_tagger(data_train,
                     [UnigramTagger, BigramTagger, TrigramTagger],
                     backoff = backoff)
print('UnigramTagger, BigramTagger, TrigramTagger')
tag.accuracy(data_test)

UnigramTagger, BigramTagger, TrigramTagger


0.827905462595221

In [15]:
tag = backoff_tagger(data_train,
                     [UnigramTagger, BigramTagger],
                     backoff = backoff)
print('UnigramTagger, BigramTagger')
tag.accuracy(data_test)

UnigramTagger, BigramTagger


0.8275343446838986

In [16]:

tag = backoff_tagger(data_train,
                     [UnigramTagger, TrigramTagger],
                     backoff = backoff)
print('UnigramTagger, TrigramTagger')
tag.accuracy(data_test)

UnigramTagger, TrigramTagger


0.8273650628296113

In [17]:

tag = backoff_tagger(data_train,
                     [ BigramTagger, TrigramTagger],
                     backoff = backoff)
print('BigramTagger, TrigramTagger')
tag.accuracy(data_test)

BigramTagger, TrigramTagger


0.1750309264926102

## Вывод
Из одиночных тегеов лучший результат показывает юниграм, Комбинации юниграма биграма и тригама как в парах так и втройках показывают схожие резудьтаты. Лучшим вариантом по точности -   Bigram and Unigram Tagger. Однако использование только Unigram Tagger будет быстрее метрика не сильно отличается. Комбинация  BigramTagger, TrigramTagger показывает неудовлетворительный результат

In [18]:
# 1.2
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [19]:
# Переведём тренировочный датасет в списки слов и списки POS-разметки
train_tok = []
train_label = []
for sent in data_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])

test_tok = []
test_label = []
for sent in data_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [47]:
train_tok[:4], train_label[:4]

(['Анкета', '.', 'Начальник', 'областного'], ['NOUN', 'PUNCT', 'NOUN', 'ADJ'])

In [20]:

test_tok = ['' if item is None else item for item in test_tok]
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

test_enc_labels = le.transform(test_label)



for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')


    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)

    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.9295527052542483
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.934859040302103
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9362588710202487


In [37]:
!pip install corus -q

In [86]:
!pip install tensorflow

In [87]:
!pip install razdel -q

In [48]:
import corus
from corus import load_ne5
import nltk
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input, Bidirectional,Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from sklearn import model_selection, preprocessing, linear_model
from razdel import tokenize

nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [70]:
import keras

In [29]:
%%capture
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

In [30]:
!unzip -q collection5.zip

replace Collection5/001.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Выявление NER с пмощью nltk

С помощью функции nltk.ne_chunk ( на уже обученном наборе с помощью nltk.pos_tag - определяет части речи) можно распознавать именованные сущности с помощью классификатора, который добавляет метки категорий, такие как PERSON, ORGANIZATION и GPE.

In [49]:
records = load_ne5('Collection5/')
document = next(records).text
document

'Д.Медведев сменил полпреда в СЗФО.\r\n\r\nПрезидент РФ Дмитрий Медведев освободил Илью Клебанова с поста полномочного представителя президента в Северо-Западном федеральном округе (СЗФО) в связи с переходом на другую работу. Об этом сообщает пресс-служба Кремля. Полпредом в СЗФО назначен Николай Винниченко.'

In [50]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }


{('Николай Винниченко', 'PERSON'),
 ('РФ Дмитрий Медведев', 'ORGANIZATION'),
 ('СЗФО', 'ORGANIZATION')}

In [53]:
document = next(records).text
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Александр', 'PERSON'),
 ('БКК', 'ORGANIZATION'),
 ('Банк', 'PERSON'),
 ('Более', 'PERSON'),
 ('Валерия Вакульчика', 'PERSON'),
 ('Валерия Иванова', 'PERSON'),
 ('Виктора Лукашенко', 'PERSON'),
 ('Вы', 'PERSON'),
 ('Иванов', 'PERSON'),
 ('Иванова мы', 'PERSON'),
 ('Кроме', 'PERSON'),
 ('Лукашенко', 'GPE'),
 ('Лукашенко', 'PERSON'),
 ('Минске', 'PERSON'),
 ('Надо', 'PERSON'),
 ('Отметим', 'PERSON'),
 ('Поэтому', 'PERSON'),
 ('России', 'PERSON'),
 ('Сергей Румас', 'PERSON'),
 ('Совмина Белоруссии', 'PERSON')}

## Выводы
#### nltk.ne_chunk имет не плохой TPR показатель однако тащит за собой много FPR обьектов

## Выявление  NER c помощью Нейросетей

In [54]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        # if "http://" in input:
        #   token='None'
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [56]:
df_words.head(5)

,word,tag
0,В,OUT
1,Европарламенте,ORG
2,избран,OUT
3,новый,OUT
4,председатель,OUT


In [88]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)


In [89]:
X_train.apply(len).max(axis=0)

55

In [90]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

train_data = train_data.batch(3056)
valid_data = valid_data.batch(3056)
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 30

## Сеть прямого распространения

In [167]:
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=seq_len
    )

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)
t=np.unique(encoder.inverse_transform(y_test),return_counts=True)[1]
t=t/t.sum()

model = tf.keras.Sequential([
  vectorize_layer,
  Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
  GlobalAveragePooling1D(),
  Dense(82, activation='relu'),
  Dropout(0.6),
  Dense(6, activation='relu'),
  ])

optim = tf.keras.optimizers.Nadam(learning_rate=0.01)
model.compile(optimizer = optim,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])
              #metrics=[tf.keras.metrics.Accuracy(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])
model.fit(train_data, validation_data=valid_data, epochs=10)

Epoch 1/10
65/65 [==============================] - 5s 66ms/step - loss: 0.9427 - accuracy: 0.8309 - val_loss: 0.6046 - val_accuracy: 0.8331
Epoch 2/10
65/65 [==============================] - 3s 47ms/step - loss: 0.9361 - accuracy: 0.6180 - val_loss: 1.3358 - val_accuracy: 0.3044
Epoch 3/10
65/65 [==============================] - 3s 48ms/step - loss: 1.0986 - accuracy: 0.4631 - val_loss: 0.9577 - val_accuracy: 0.5184
Epoch 4/10
65/65 [==============================] - 6s 93ms/step - loss: 0.7833 - accuracy: 0.6331 - val_loss: 0.6655 - val_accuracy: 0.6865
Epoch 5/10
65/65 [==============================] - 4s 59ms/step - loss: 0.6693 - accuracy: 0.6873 - val_loss: 0.6492 - val_accuracy: 0.6946
Epoch 6/10
65/65 [==============================] - 3s 53ms/step - loss: 0.6088 - accuracy: 0.7169 - val_loss: 0.6842 - val_accuracy: 0.6716
Epoch 7/10
65/65 [==============================] - 4s 56ms/step - loss: 0.6392 - accuracy: 0.6889 - val_loss: 0.6937 - val_accuracy: 0.6655
Epoch 8/10
65

In [168]:
y_pred = model.predict(X_test)
predicted_categories = np.argmax(y_pred, axis=1)

In [169]:
 predicted_categories, y_test

(array([4, 4, 4, ..., 4, 4, 4]), array([4, 5, 4, ..., 4, 4, 4]))

In [170]:
len(predicted_categories), len(X_test)

(65903, 65903)

In [171]:
clr = classification_report(y_test, predicted_categories)
print( clr);

              precision    recall  f1-score   support

           0       0.08      0.95      0.15      1096
           1       0.85      0.04      0.07      1098
           2       0.00      0.00      0.00       592
           3       0.92      0.04      0.08      3383
           4       0.94      0.90      0.92     54451
           5       0.98      0.13      0.22      5283

    accuracy                           0.77     65903
   macro avg       0.63      0.34      0.24     65903
weighted avg       0.92      0.77      0.79     65903



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Рекурентная сеть

In [163]:
model = tf.keras.Sequential([
    vectorize_layer,
    Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
    Bidirectional(tf.keras.layers.LSTM(64)),
    Dense(64, activation='relu'),
    Dropout(0.6),
    Dense(6, activation='softmax')
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_data, validation_data=valid_data, epochs=10)

Epoch 1/10
65/65 [==============================] - 161s 2s/step - loss: 1.1105 - accuracy: 0.8113 - val_loss: 0.4965 - val_accuracy: 0.8282
Epoch 2/10
65/65 [==============================] - 116s 2s/step - loss: 0.4303 - accuracy: 0.8493 - val_loss: 0.3877 - val_accuracy: 0.8866
Epoch 3/10
65/65 [==============================] - 115s 2s/step - loss: 0.3139 - accuracy: 0.9006 - val_loss: 0.2988 - val_accuracy: 0.9051
Epoch 4/10
65/65 [==============================] - 115s 2s/step - loss: 0.2349 - accuracy: 0.9256 - val_loss: 0.2569 - val_accuracy: 0.9251
Epoch 5/10
65/65 [==============================] - 117s 2s/step - loss: 0.1909 - accuracy: 0.9413 - val_loss: 0.2395 - val_accuracy: 0.9304
Epoch 6/10
65/65 [==============================] - 116s 2s/step - loss: 0.1682 - accuracy: 0.9495 - val_loss: 0.2308 - val_accuracy: 0.9380
Epoch 7/10
65/65 [==============================] - 117s 2s/step - loss: 0.1532 - accuracy: 0.9560 - val_loss: 0.2246 - val_accuracy: 0.9401
Epoch 8/10
65

In [165]:
y_pred = model.predict(X_test)
predicted_categories = np.argmax(y_pred, axis=1)

In [166]:

clr = classification_report(y_test, predicted_categories)
print( clr);

              precision    recall  f1-score   support

           0       0.90      0.90      0.90      1096
           1       0.85      0.77      0.81      1098
           2       0.91      0.77      0.83       592
           3       0.91      0.54      0.67      3383
           4       0.94      1.00      0.97     54451
           5       0.99      0.70      0.82      5283

    accuracy                           0.94     65903
   macro avg       0.92      0.78      0.83     65903
weighted avg       0.94      0.94      0.94     65903



## Выводы
### рекурентная сеть лучше справляется с задачей NER. В целом сеть можно еще дообучить. Переобучения что в сети прямого распостранения что в рекурентной не наблюдается. Сеть прямого распостранения плохо класифицирует 2 класс